In [17]:
import PyPDF2
import os
import fitz  # PyMuPDF for PDF text extraction
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
import pinecone
from tqdm.autonotebook import tqdm
from dotenv import load_dotenv,find_dotenv
from langchain.vectorstores import Pinecone
from tqdm.autonotebook import tqdm

In [73]:
def extract_text_from_pdf(pdf_path):
    # Extract text from the PDF using PyMuPDF (Fitz)
    doc = fitz.open(pdf_path)
    text = ''
    for page_num in range(doc.page_count):
        page = doc[page_num]
        text += page.get_text()
        # print(text)
    doc.close()
    return text

# Replace with the path to your PDF files directory
pdf_files_directory = 'Data/pdf'

# List all PDF files in the directory
pdf_files = [os.path.join(pdf_files_directory, filename) for filename in os.listdir(pdf_files_directory) if filename.endswith('.pdf')]

all_pdf_text=[]
# Iterate through each PDF file
for pdf_file in pdf_files:
    print(pdf_file)
    # Extract text from the PDF
    pdf_text = extract_text_from_pdf(pdf_file)
    all_pdf_text.append(pdf_text)
# Join the PDF text with commas as separators
all_pdf_text_with_comma = ','.join(all_pdf_text)

print(all_pdf_text_with_comma)
# print(all_pdf_text)



Data/pdf\Academic Integrity_ ORG04655 _ MS-BAIS Current Students.pdf
Data/pdf\Academic Probation_ ORG04655 _ MS-BAIS Current Students.pdf
Data/pdf\Canned Responses re Applicants and Potential Applicants.pdf
Data/pdf\Course Registration_ ORG04655 _ MS-BAIS Current Students.pdf
Data/pdf\CPT Internship FAQ_ ORG04655 _ MS-BAIS Current Students.pdf
Data/pdf\CPT Internships_ ORG04655 _ MS-BAIS Current Students.pdf
Data/pdf\Graduate_Teaching Assistantships_ ORG04655 _ MS-BAIS Current Students.pdf
Data/pdf\How to apply for graduation_ ORG04655 _ MS-BAIS Current Students.pdf
Data/pdf\Independent Study_ ORG04655 _ MS-BAIS Current Students.pdf
Data/pdf\MS-BAIS Advising FAQ_ ORG04655 _ MS-BAIS Current Students.pdf
Data/pdf\Practice Center Projects_ ORG04655 _ MS-BAIS Current Students.pdf
Data/pdf\Professional Development Certificates_ ORG04655 _ MS-BAIS Current Students.pdf
Data/pdf\Reduced Course Loads_ ORG04655 _ MS-BAIS Current Students.pdf
Data/pdf\Transfer Credits_ ORG04655 _ MS-BAIS Current 

In [72]:
# Import utility for splitting up texts and split up the explanation given above into document chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 0,
)

In [75]:
# # Split the text into chunks using RecursiveCharacterTextSplitter
chunks = text_splitter.create_documents([all_pdf_text_with_comma])

# # Initialize an empty list to store chunks
# all_chunks = []

# # Iterate through each PDF text
# for pdf_text in all_pdf_text_with_comma:
#     # Split the text into chunks using RecursiveCharacterTextSplitter
#     chunks = text_splitter.create_documents(pdf_text)
#     all_chunks.extend(chunks)
#  # Print or process the chunks as needed
for chunk in chunks:
    print(chunk)

page_content='10/14/23, 9:52 PM\nAcademic Integrity: ORG04655 : MS-BAIS Current Students\nhttps://usflearn.instructure.com/courses/1794296/pages/academic-integrity?module_item_id=28098693\n1/3\nAcademic Integrity\nThe MS-BAIS program follow USF\'s official policies on academic integrity, which are described in\ndetail at https://www.usf.edu/undergrad/students/ethics-integrity.aspx\n(https://www.usf.edu/undergrad/students/ethics-integrity.aspx) . We encourage you to read through this\npage to familiarize yourself of our official policy and consequences for violations. Students are\nroutinely expelled from USF (without degree) every year for violations of our official academic\nintegrity policies. "I did not know" or "This is not considered plagiarism in the culture where I\'m from"\nwill not be considered an acceptable excuse if you happen to be involved in any of these\ntransgressions. \n \nWhat are violations of academic integrity?'
page_content="Cheating: Cheating is using or attempt

In [76]:
embeddings = OpenAIEmbeddings(deployment="text-embedding-ada-002")

In [77]:
# Import and initialize Pinecone client
# You will need to create a Pinecone account and create an API key and environment variable, see www.pinecone.io


pinecone.init(
    api_key=os.getenv('PINECONE_API_KEY'),
    environment=os.getenv('PINECONE_ENV')
)

In [78]:
pinecone.delete_index("langchain-demo-index")

In [80]:
indexes_info = pinecone.list_indexes()
print("Indexes:", indexes_info)

Indexes: ['langchain-demo-index']


In [79]:
# pinecone.init(api_key=os.getenv('PINECONE_API_KEY'), environment=os.getenv('PINECONE_ENV'))
pinecone.create_index("langchain-demo-index", dimension=1536)

In [81]:
# Upload vectors to Pinecone
# You need first create an index in Pinecone, see www.pinecone.io - set it to 1536 dimensions

index_name = "langchain-demo-index"
# flattened_data = [doc for sublist in all_chunks for doc in sublist]
# search = Pinecone.from_documents(flattened_data, embeddings, index_name=index_name)
search = Pinecone.from_documents(chunks , embeddings, index_name=index_name) # load all text chunks into Pinecone with the associated embeddings

In [83]:
# Do a simple vector similarity search
from pprint import pprint
query = "what is pcp?"
result = search.similarity_search(query)

pprint(result)

[Document(page_content="Unlike internships, where you have to get your own internship, PCP is arranged by SISM. Corporate\nemployers approach SISM with their projects every semester. Anna sends out e-mails to students\nindicating these projects usually before the start of Fall and Spring semesters. You can send in your\nresumes to Anna, indicating which PCPs you are interest it. The corporate sponsor will review all\nresumes and shortlist candidates for interview. Following your interviews, candidates will be notified\nof their interview results. Make sure that you are on Anna's e-mail list to receive these PCP e-mails.\n,10/14/23, 9:53 PM\nProfessional Development Certificates: ORG04655 : MS-BAIS Current Students\nhttps://usflearn.instructure.com/courses/1794296/pages/professional-development-certificates?module_item_id=28098697\n1/1\nProfessional Development Certificates\nTo help our students in their efforts to acquire professionally marketable knowledge and skills, the"),
 Document